# Whisper for Inferentia2

This sample shows how to compile & run Whisper models (different sizes) on Inferentia2. It makes use of the HF weights:  
  - Tiny: https://huggingface.co/openai/whisper-tiny
  - Small: https://huggingface.co/openai/whisper-small
  - Medium: https://huggingface.co/openai/whisper-medium
  - Large-v3: https://huggingface.co/openai/whisper-large-v3

Given the largest model has only 1.5B params, it fits into just 1 core when quantized to bf16. Also, this model is an encoder-decoder, so the strategy is to compile both components individually and then put them back into the original model structure. After that, both encoder and decoder will be accelerated on inf2.

You can use the smallest instance for this experiment: inf2.xlarge, but to achieve a higher througput by launching multiple copies of the model to serve clients in parallel, it is recommended to use a larger instance like ml.inf2.24xlarge or trn1.32xlarge.

Follow the [instructions from this page to setup the environment.](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx) It is recommended the usage of the following container (DLC) to run your experiments: **Deep Learning Container**: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.19.1-ubuntu20.04

This guarantees you'll be using the exact same libraries of this experimentation.

Also, make sure you install the following additional libraries in your environment. Pay attention to the transformers version, newer versions might not work.

## Install Dependencies
This tutorial requires the following pip packages:

- `transformers==4.36.2`
- `soundfile==0.12.1`
- `datasets==2.18.0`
- `librosa==0.10.1`

In [ ]:
%pip install -U transformers==4.36.2 datasets==2.18.0 soundfile==0.12.1 librosa==0.10.1

In [ ]:
import os
os.environ['NEURON_RT_NUM_CORES']='2'
import types
import torch
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration


# please, start by selecting the desired model size
suffix="tiny"
#suffix="small"
#suffix="medium"
#suffix="large-v3"
model_id=f"openai/whisper-{suffix}"

# this will load the tokenizer + two copies of the model. cpu_model will be used later for results comparison
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id, torchscript=True)
cpu_model = WhisperForConditionalGeneration.from_pretrained(model_id, torchscript=True)

# Load a sample from the dataset
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# sample #3 is ~9.9seconds and produces 33 output tokens + pad token
sample = dataset[3]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

# output_attentions is required if you want to return word timestamps
# if you don't need timestamps, just set this to False and get some better latency
output_attentions=False

batch_size=1
# this is the maximum number of tokens the model will be able to decode
# for the sample #3 we selected above, this is enough. If you're planning to 
# process larger samples, you need to adjust it accordinly.
max_dec_len = 64
# num_mel_bins,d_model --> these parameters where copied from model.conf (found on HF repo)
# we need them to correctly generate dummy inputs during compilation
dim_enc=model.config.num_mel_bins
dim_dec=model.config.d_model
print(f'Dim enc: {dim_enc}; Dim dec: {dim_dec}')

In [ ]:
import types
import torch.nn.functional as F
from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions,BaseModelOutput

# Now we need to simplify both encoding & decoding forward methods to make them 
# compilable. Please notice that these methods overwrite the original ones, but
# keeps retro-compatibility. Also, we'll use use a new variable "forward_neuron"
# to invoke the model on inf2
def enc_f(self, input_features, attention_mask, **kwargs):
    if hasattr(self, 'forward_neuron'):
        out = self.forward_neuron(input_features, attention_mask)
    else:
        out = self.forward_(input_features, attention_mask, return_dict=True)
    return BaseModelOutput(**out)

def dec_f(self, input_ids, attention_mask=None, encoder_hidden_states=None, **kwargs):
    out = None        
    if not attention_mask is None and encoder_hidden_states is None:
        # this is a workaround to align the input parameters for NeuronSDK tracer
        # None values are not allowed during compilation
        encoder_hidden_states, attention_mask = attention_mask,encoder_hidden_states
    inp = [input_ids, encoder_hidden_states]
    
    # pad the input to max_dec_len
    if inp[0].shape[1] > self.max_length:
        raise Exception(f"The decoded sequence is not supported. Max: {self.max_length}")
    pad_size = torch.as_tensor(self.max_length - inp[0].shape[1])
    inp[0] = F.pad(inp[0], (0, pad_size), "constant", processor.tokenizer.pad_token_id)
    
    if hasattr(self, 'forward_neuron'):
        out = self.forward_neuron(*inp)
    else:
        # output_attentions is required if you want timestamps
        out = self.forward_(input_ids=inp[0], encoder_hidden_states=inp[1], return_dict=True, use_cache=False, output_attentions=output_attentions)
    # unpad the output
    out['last_hidden_state'] = out['last_hidden_state'][:, :input_ids.shape[1], :]
    # neuron compiler doesn't like tuples as values of dicts, so we stack them into tensors
    # also, we need to average axis=2 given we're not using cache (use_cache=False)
    # that way, to avoid an issue with the pipeline we change the shape from:
    #  bs,num selected,num_tokens,1500 --> bs,1,num_tokens,1500
    # I suspect there is a bug in the HF pipeline code that doesn't support use_cache=False for
    # word timestamps, that's why we need that.
    if not out.get('attentions') is None:
        out['attentions'] = torch.stack([torch.mean(o[:, :, :input_ids.shape[1], :input_ids.shape[1]], axis=2, keepdim=True) for o in out['attentions']])
    if not out.get('cross_attentions') is None:
        out['cross_attentions'] = torch.stack([torch.mean(o[:, :, :input_ids.shape[1], :], axis=2, keepdim=True) for o in out['cross_attentions']])
    return BaseModelOutputWithPastAndCrossAttentions(**out)

if not hasattr(model.model.encoder, 'forward_'): model.model.encoder.forward_ = model.model.encoder.forward
if not hasattr(model.model.decoder, 'forward_'): model.model.decoder.forward_ = model.model.decoder.forward

model.model.encoder.forward = types.MethodType(enc_f, model.model.encoder)
model.model.decoder.forward = types.MethodType(dec_f, model.model.decoder)

model.model.decoder.max_length = max_dec_len

In [ ]:
# warmup model
y1 = model.generate(input_features)

## Trace Encoder

In [ ]:
import os
import torch
import torch_neuronx

model_filename=f"whisper_{suffix}_{batch_size}_neuron_encoder.pt"
if not os.path.isfile(model_filename):
    inp = (torch.zeros([1, dim_enc, 3000], dtype=torch.float32), torch.zeros([1, dim_enc], dtype=torch.int64))
    if hasattr(model.model.encoder, 'forward_neuron'): del model.model.encoder.forward_neuron
    neuron_encoder = torch_neuronx.trace(
        model.model.encoder, 
        inp,
        compiler_args='--model-type=transformer --auto-cast=all --auto-cast-type=bf16',
        compiler_workdir='./enc_dir',      
        inline_weights_to_neff=False)
    neuron_encoder.save(model_filename)
    model.model.encoder.forward_neuron = neuron_encoder
else:
    model.model.encoder.forward_neuron = torch.jit.load(model_filename)

## Trace decoder

In [ ]:
import torch
import torch_neuronx

model_filename=f"whisper_{suffix}_{batch_size}_{max_dec_len}_neuron_decoder.pt"
if not os.path.isfile(model_filename):
    inp = (torch.zeros([1, max_dec_len], dtype=torch.int64), torch.zeros([1, 1500, dim_dec], dtype=torch.float32))
    if hasattr(model.model.decoder, 'forward_neuron'): del model.model.decoder.forward_neuron
    neuron_decoder = torch_neuronx.trace(
        model.model.decoder, 
        inp,
        compiler_args='--model-type=transformer --auto-cast=all --auto-cast-type=bf16',
        compiler_workdir='./dec_dir',      
        inline_weights_to_neff=True)
    neuron_decoder.save(model_filename)
    model.model.decoder.forward_neuron = neuron_decoder
else:
    model.model.decoder.forward_neuron = torch.jit.load(model_filename)

## Test

In [ ]:
# warmup inf2 model
y1 = model.generate(input_features)

In [26]:
import time
t=time.time()
y1 = model.generate(input_features)
print(f"Elapsed inf2: {time.time()-t}")
t=time.time()
y2 = cpu_model.generate(input_features)
print(f"Elapsed cpu: {time.time()-t}")
print(f"Tokens inf2: {y1}")
print(f"Tokens cpu: {y2}")
t1 = processor.batch_decode(y1, skip_special_tokens=True)
t2 = processor.batch_decode(y2, skip_special_tokens=True)
print(f"Out inf2: {t1}")
print(f"Out cpu: {t2}")

Elapsed inf2: 0.08130168914794922
Elapsed cpu: 1.1965036392211914
Tokens inf2: tensor([[50258, 50259, 50359, 50363,   634,   575, 12525, 22618,  1968,  6144,
         35617, 20084,  1756,   311,   589,   307,   534, 10281,   934,   439,
           293,   393,  4411,   294,   309,   457,   707,   295, 26916,   286,
           392,  6628,    13, 50257]])
Tokens cpu: tensor([[50258, 50259, 50359, 50363,   634,   575, 12525, 22618,  1968,  6144,
         35617, 20084,  1756,   311,   589,   307,   534, 10281,   934,   439,
           293,   393,  4411,   294,   309,   457,   707,   295, 26916,   286,
           392,  6628,    13, 50257]])
Out inf2: [" He has grave doubts whether Sir Frederick Layton's work is really Greek after all and can discover in it but little of Rocky Ithaca."]
Out cpu: [" He has grave doubts whether Sir Frederick Layton's work is really Greek after all and can discover in it but little of Rocky Ithaca."]


## Pipeline Mode

In [ ]:
import torch
import torch_neuronx
from datasets import load_dataset
from transformers import pipeline, WhisperProcessor

if not output_attentions:
    raise Exception("Word timestamp not supported. Please set output_attentions=True and recompile the model")

pipe = pipeline(
  "automatic-speech-recognition",
  model=model_id,
  chunk_length_s=30
)
pipe.model = model
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[3]["audio"]

# we can also return timestamps for the predictions
## Option return_timestamps can be: True, False, "word" or "char"
t=time.time()
prediction = pipe(sample.copy(), batch_size=1, return_timestamps="word")["chunks"]
print(f"Elapsed: {time.time()-t}")
for p in prediction:
    print(p)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Elapsed: 0.1590416431427002
{'text': ' He', 'timestamp': (0.0, 0.72)}
{'text': ' has', 'timestamp': (0.72, 1.0)}
{'text': ' grave', 'timestamp': (1.0, 1.36)}
{'text': ' doubts', 'timestamp': (1.36, 1.7)}
{'text': ' whether', 'timestamp': (1.7, 2.06)}
{'text': ' Sir', 'timestamp': (2.06, 2.48)}
{'text': ' Frederick', 'timestamp': (2.48, 2.82)}
{'text': " Layton's", 'timestamp': (2.82, 3.36)}
{'text': ' work', 'timestamp': (3.36, 3.72)}
{'text': ' is', 'timestamp': (3.72, 3.98)}
{'text': ' really', 'timestamp': (3.98, 4.48)}
{'text': ' Greek', 'timestamp': (4.48, 4.86)}
{'text': ' after', 'timestamp': (4.86, 5.3)}
{'text': ' all', 'timestamp': (5.3, 6.0)}
{'text': ' and', 'timestamp': (6.0, 6.36)}
{'text': ' can', 'timestamp': (6.36, 6.54)}
{'text': ' discover', 'timestamp': (6.54, 6.98)}
{'text': ' in', 'timestamp': (6.98, 7.18)}
{'text': ' it', 'timestamp': (7.18, 7.38)}
{'text': ' but', 'timestamp': (7.38, 7.6)}
{'text': ' little', 'timestamp': (7.6, 7.96)}
{'text': ' of', 'timestamp'